In [1]:
import sys
from pathlib import Path
repo_root = Path.cwd().parent
sys.path.insert(0, str(repo_root / "src"))

import numpy as np
import pandas as pd
import random
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

from GurobiParamEnv import InexactGBDEnv

from policies import PPOPolicy, OptimalPolicy, RandomPolicy, ExponentialPolicy

In [2]:
def make_env():
    return InexactGBDEnv()

# 1) Create and load the VecNormalize-wrapped env once
raw_env = DummyVecEnv([make_env])
env = VecNormalize.load("../models/vecnormalize_benders.pkl", raw_env)
env.training = False    # freeze running stats
env.norm_reward = False # use true rewards

# 2) Load trained PPO
model = PPO.load("../models/ppo_benders_model", env=env)

# 3) Evaluation helper that seeds both numpy/python env and SB3
def evaluate(agent, env, seed=0):
    metrics = {"reward": [], "mp_time": [], "sp_time": [], 
               "iterations": [], "true_gap": [], "master_gap_tol": []}

    np.random.seed(seed)
    random.seed(seed)

    obs = env.reset()
    done = False
    total_rew = 0.0

    actions = []
    while not done:
        action, _ = agent.predict(obs, deterministic=True)
        
        obs, reward, done, info = env.step(action)
        total_rew += reward[0]
        actions.append(action)
        metrics["reward"].append(total_rew)
        metrics["mp_time"].append(info[0]["mp_time"])
        metrics["sp_time"].append(info[0]["sp_time"])
        metrics["iterations"].append(info[0]["iteration"])
        metrics["true_gap"].append(info[0]["true gap"])
        metrics["master_gap_tol"].append(info[0]["master_gap_tol"])

    return metrics, actions

Configuring transition blocks...
Configuring initial transition blocks...
Configuring scheduling block...
Adding noise...
In scheduling_opt, calculating transition time bounds...
In scheduling_opt, calculating initial transition time bounds...
Configuring transition cost bounds...
Configuring initial transition cost bounds...


In [3]:
rl_policy = PPOPolicy(model, env)
rl_metrics, rl_actions = evaluate(rl_policy, env)

Configuring transition blocks...
Configuring initial transition blocks...
Configuring scheduling block...
Adding noise...
In scheduling_opt, calculating transition time bounds...
In scheduling_opt, calculating initial transition time bounds...
Configuring transition cost bounds...
Configuring initial transition cost bounds...
raw index: -0.19999999999999996
true gap: 0.9999999970866963
tolerance [0.3]
Iteration: 1
Set parameter Username
Set parameter LicenseID to value 2589360
Academic license - for non-commercial use only - expires 2025-11-23
        0.00 seconds required for presolve
        0.03 seconds required for solver
        0.00 seconds required for postsolve
raw index: -0.8
true gap: 1.1467905276593253
tolerance [0.11557905276593251]
Iteration: 2
        0.00 seconds required for presolve
        0.03 seconds required for solver
        0.00 seconds required for postsolve
raw index: 0.0
true gap: 0.4253094578057463
tolerance [0.21315472890287315]
Iteration: 3
        0.00 se

In [4]:
baseline_opt = OptimalPolicy(env)
base_opt_metrics, base_opt_actions = evaluate(baseline_opt, env)


Configuring transition blocks...
Configuring initial transition blocks...
Configuring scheduling block...
Adding noise...
In scheduling_opt, calculating transition time bounds...
In scheduling_opt, calculating initial transition time bounds...
Configuring transition cost bounds...
Configuring initial transition cost bounds...
raw index: -1.0
true gap: 0.9999999970866963
tolerance [0.001]
Iteration: 1
        0.00 seconds required for presolve
        0.09 seconds required for solver
        0.00 seconds required for postsolve
raw index: -1.0
true gap: 0.4813094218067998
tolerance [0.001]
Iteration: 2
        0.00 seconds required for presolve
        0.06 seconds required for solver
        0.00 seconds required for postsolve
raw index: -1.0
true gap: 0.37150511220186877
tolerance [0.001]
Iteration: 3
        0.00 seconds required for presolve
        0.08 seconds required for solver
        0.00 seconds required for postsolve
raw index: -1.0
true gap: 0.37143644730204006
tolerance [0.

In [5]:
baseline_rand = RandomPolicy(env) # 0.001
base_rand_metrics, base_rand_actions = evaluate(baseline_rand, env)

Configuring transition blocks...
Configuring initial transition blocks...
Configuring scheduling block...
Adding noise...
In scheduling_opt, calculating transition time bounds...
In scheduling_opt, calculating initial transition time bounds...
Configuring transition cost bounds...
Configuring initial transition cost bounds...
raw index: -0.19999999999999996
true gap: 0.9999999970866963
tolerance [0.3]
Iteration: 1
        0.00 seconds required for presolve
        0.02 seconds required for solver
        0.00 seconds required for postsolve
raw index: 0.3999999999999999
true gap: 1.1467905276593253
tolerance [0.3]
Iteration: 2
        0.00 seconds required for presolve
        0.02 seconds required for solver
        0.00 seconds required for postsolve
raw index: 0.19999999999999996
true gap: 0.8818625851993578
tolerance [0.3]
Iteration: 3
        0.00 seconds required for presolve
        0.03 seconds required for solver
        0.00 seconds required for postsolve
raw index: 0.60000000

In [6]:
baseline_exp = ExponentialPolicy(env, 0.3, 0.8)
base_exp_metrics, base_exp_actions = evaluate(baseline_exp, env)

Configuring transition blocks...
Configuring initial transition blocks...
Configuring scheduling block...
Adding noise...
In scheduling_opt, calculating transition time bounds...
In scheduling_opt, calculating initial transition time bounds...
Configuring transition cost bounds...
Configuring initial transition cost bounds...
Iteration: 1
        0.00 seconds required for presolve
        0.02 seconds required for solver
        0.00 seconds required for postsolve
Iteration: 2
        0.00 seconds required for presolve
        0.02 seconds required for solver
        0.00 seconds required for postsolve
Iteration: 3
        0.00 seconds required for presolve
        0.03 seconds required for solver
        0.00 seconds required for postsolve
Iteration: 4
        0.00 seconds required for presolve
        0.04 seconds required for solver
        0.00 seconds required for postsolve
Iteration: 5
        0.00 seconds required for presolve
        0.03 seconds required for solver
        0.0

In [7]:
# df = pd.read_csv("test_episode_data.csv")
# base_rand_metrics = df[df['policy'] == 'Random']
# base_exp_metrics = df[df['policy'] == 'Exponential']
# base_opt_metrics = df[df['policy'] == 'Optimal']
# rl_metrics = df[df['policy'] == 'RL']
# df

In [8]:
import plotly.graph_objects as go

# 1) compute n and pad helper
n = max(
    len(rl_metrics['mp_time']),
    len(base_opt_metrics['mp_time']),
    len(base_rand_metrics['mp_time']),
    len(base_exp_metrics['mp_time'])
)
def pad_with_nan(lst, length):
    arr = np.array(lst, dtype=float).reshape(-1)
    return np.concatenate([arr, np.full(max(0, length - arr.size), np.nan)])

rl_mp_p        = pad_with_nan(rl_metrics['mp_time'], n)
base_opt_mp_p  = pad_with_nan(base_opt_metrics['mp_time'], n)   # “Baseline”
base_rand_mp_p = pad_with_nan(base_rand_metrics['mp_time'], n)
base_exp_mp_p = pad_with_nan(base_exp_metrics['mp_time'], n)   # “Constant”

# 2) your color map
color_map = {
    'DRL':        'rgb(255,127,14)',
    'Random':     'rgb(44,160,44)',
    'Exponential':'rgb(148,103,189)',
    'Optimal':    'rgb(31,119,180)',
}

# 3) x‐axis (iterations)
iterations = list(range(n))

# 4) build figure
fig = go.Figure([
    go.Bar(name='DRL',
           x=iterations, y=rl_mp_p,
           marker_color=color_map['DRL'], 
           opacity=0.8),
    go.Bar(name='Random',
           x=iterations, y=base_rand_mp_p,
           marker_color=color_map['Random'],
           opacity=0.8),
    go.Bar(name='Exponential',
           x=iterations, y=base_exp_mp_p,
           marker_color=color_map['Exponential'],
           opacity=0.8),
    go.Bar(name='Optimal',
           x=iterations, y=base_opt_mp_p,
           marker_color=color_map['Optimal'],
           opacity=0.8)
    
])

fig.update_layout(
    barmode='group',
    font=dict(
        family="Times New Roman",  # global font
        size=16
    ),
    xaxis=dict(
        title='Iteration',
        title_font=dict(family="Times New Roman", size=20),
        tickfont=dict(family="Times New Roman", size=18)
    ),
    yaxis=dict(
        title='Master problem runtime (s)',
        title_font=dict(family="Times New Roman", size=20),
        tickfont=dict(family="Times New Roman", size=18)
    ),
    legend=dict(
        orientation='h',
        yanchor='bottom', y=1.02,
        xanchor='center', x=0.5,
        font=dict(family="Times New Roman", size=18)
    ),
    width=1100, height=450,
    margin=dict(l=80, r=20, t=50, b=60)
)

fig.show()

In [9]:
# fig.write_image("./plots/per_itr_runtime.png", width=1100, height=450, scale=3)

In [10]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# --- your existing color_map and metrics dicts must be defined above ---
color_map = {
    'RL':        'rgb(255,127,14)',
    'Random':     'rgb(44,160,44)',
    'Exponential':'rgb(148,103,189)',
    'Optimal':    'rgb(31,119,180)',
}

marker_map = {
    'RL':          'star',
    'Random':      'square',
    'Exponential': 'diamond',
    'Optimal':     'circle'
}


label_map = {
    'RL':          'RL-iGBD',
    'Random':      'Rand-iGBD',
    'Exponential': 'Exp-iGBD',
    'Optimal':     'GBD'
}

# --- compute cumulative master‐problem runtimes ---
cum_mp_rl    = np.cumsum(rl_metrics   ['mp_time'])
cum_mp_opt   = np.cumsum(base_opt_metrics['mp_time'])   # Baseline
cum_mp_exp   = np.cumsum(base_exp_metrics['mp_time'])   # Constant
cum_mp_rand  = np.cumsum(base_rand_metrics['mp_time'])  # Random

# --- make 3×1 subplots, sharing the x‐axis ---
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.09,
)

# helper to add your original two panels
def add_policy_trace(policy, metrics, row, y_key, log_scale=False, show_legend=False):
    fig.add_trace(
        go.Scatter(
            x=list(range(len(metrics[y_key]))),
            y=metrics[y_key],
            mode="lines+markers",
            name=label_map[policy],
            legendgroup=policy,
            showlegend=show_legend,
            line_color=color_map[policy],
            marker_color=color_map[policy],
            marker_symbol=marker_map[policy],
            opacity=0.8
        ),
        row=row, col=1
    )
    if log_scale:
        fig.update_yaxes(type="log", row=row, col=1)

# Row 1: Master gap tolerance (with legend)
for policy, metrics in [
    ("RL",        rl_metrics),
    ("Random",     base_rand_metrics),
    ("Exponential",base_exp_metrics),
    ("Optimal",    base_opt_metrics),
]:
    add_policy_trace(policy, metrics, row=1, y_key="master_gap_tol", show_legend=True)

# Row 2: Benders (true) gap, log‐scale (no legend)
for policy, metrics in [
    ("RL",        rl_metrics),
    ("Random",     base_rand_metrics),
    ("Exponential",base_exp_metrics),
    ("Optimal",    base_opt_metrics),
]:
    add_policy_trace(policy, metrics, row=2, y_key="true_gap", log_scale=True, show_legend=False)

# Row 3: Cumulative master‐problem runtime (no legend)
for policy, cum_data in [
    ("RL",       cum_mp_rl),
    ("Random",    cum_mp_rand),
    ("Exponential",  cum_mp_exp),
    ("Optimal",  cum_mp_opt),

]:
    # note: map “Baseline”→Optimal color, “Constant”→Exponential color
    color_key = {
      "RL": "RL",
      "Random": "Random",
      "Exponential": "Exponential",
      "Optimal": "Optimal"
    }[policy]
    fig.add_trace(
        go.Scatter(
            x=list(range(0, len(cum_data))),
            y=cum_data,
            mode="lines+markers",
            name=label_map[policy],
            legendgroup=policy,
            showlegend=False,
            line_color=color_map[color_key],
            marker_color=color_map[color_key],
            marker=dict(
            symbol=marker_map[policy],
            color= color_map[policy],
            # size=8
        ),
            opacity=0.9
        ),
        row=3, col=1
    )

# --- Y‐axis styling ---
fig.update_yaxes(
    title_text="Master gap tolerance",
    row=1, col=1,
    title_font=dict(family="Times New Roman", size=18),
    tickfont=dict(family="Times New Roman", size=16),
    tickformat=".2f"
    
)
fig.update_yaxes(
    title_text="Benders gap",
    row=2, col=1,
    title_font=dict(family="Times New Roman", size=18),
    tickfont=dict(family="Times New Roman", size=16),
    showgrid=True,
    type="log",
    tickmode="auto",
    nticks=5,
    # force "power" exponent format: 10^{…}
    exponentformat="power",
    showexponent="all"
)
fig.update_yaxes(
    title_text="Cumulative master runtime (s)",
    row=3, col=1,
    title_font=dict(family="Times New Roman", size=18),
    tickfont=dict(family="Times New Roman", size=16),
    showgrid=True,
    tickformat=".2f"
)

# --- X‐axis styling (iteration & ticks on all rows) ---
for r in [1,2,3]:
    fig.update_xaxes(
        title_text="Iteration",
        row=r, col=1,
        gridcolor='lightgrey',
        gridwidth=1,
        title_font=dict(family="Times New Roman", size=18),
        tickfont=dict(family="Times New Roman", size=16),
        ticks="inside",
        mirror=True,
        showticklabels=True,
        tickvals=[0, 4, 8, 12, 16, 20, 24],
    )

fig.update_xaxes(
    showgrid=True,
    gridcolor='lightgrey',
    gridwidth=1,
    zeroline=False,
    showline=True,
    mirror=True,
    linecolor='black',
    ticks='outside',
    title_standoff=5
)
fig.update_yaxes(
    showgrid=True,
    gridcolor='lightgrey',
    gridwidth=1,
    zeroline=False,
    mirror=True,
    showline=True,
    linecolor='black',
    ticks='outside'
)

# --- Legend & global layout ---
fig.update_layout(
    font=dict(family="Times New Roman", color="black", size=16),
    legend=dict(
        orientation="v",       # vertical layout
        x=0.97,                # near the right edge
        y=0.992,                # near the top edge
        xanchor="right",       
        yanchor="top",
        bordercolor="black",   # frame color
        borderwidth=1          # frame width
    ),
    width=500,
    height=1000,
    paper_bgcolor='white',
    plot_bgcolor='white',
    margin=dict(l=60, r=10, t=10, b=10)
)

fig.show()

In [11]:
# fig.write_image("benders_convergence.pdf", format="pdf")

### Use this to save the test results.

In [12]:
# records = []
# policies = [
#     ('RL',          rl_metrics,   cum_mp_rl),
#     ('Random',      base_rand_metrics, cum_mp_rand),
#     ('Exponential', base_exp_metrics,  cum_mp_exp),
#     ('Optimal',     base_opt_metrics,  cum_mp_opt)
# ]

# for policy, metrics, cum in policies:
#     n = len(metrics['mp_time'])
#     master_gaps = metrics.get('master_gap_tol', [None]*n)
#     true_gaps   = metrics.get('true_gap',      [None]*n)

#     for i in range(n):
#         records.append({
#             'policy':         policy,
#             'iteration':      i,
#             'master_gap_tol': master_gaps[i],
#             'true_gap':       true_gaps[i],
#             'mp_time':        metrics['mp_time'][i],
#             'cum_mp_time':    cum[i]
#         })

# df_metrics = pd.DataFrame.from_records(records)
# df_metrics.to_csv('test_episode_data.csv', index=False)

# print(f"Saved combined metrics to metrics_data.csv ({len(df_metrics)} rows)")

In [13]:
# fig.write_image("benders_convergence_sp.png", scale=5)

In [14]:
print('Master problem runtime comparison:')
print('    RL:', sum(rl_metrics['mp_time']))
print('    Constant:', sum(base_opt_metrics['mp_time']))
print('    Random:', sum(base_rand_metrics['mp_time']))
print('    Exponential:', sum(base_exp_metrics['mp_time']))

Master problem runtime comparison:
    RL: 1.4549310207366943
    Constant: 2.8387084007263184
    Random: 2.1872613430023193
    Exponential: 2.5817551612854004
